# Downstream Analysis Pipeline

This notebook demonstrates the complete downstream analysis workflow for segmented multiplex tissue imaging data:

1. **Feature Extraction** - Extract morphology and marker intensities from segmented cells
2. **Preprocessing** - Scale, filter, and clean the feature data
3. **Clustering** - PCA, Harmony batch correction, and Louvain clustering
4. **Visualization** - Heatmaps, UMAP plots, and composition analysis
5. **QuPath Export** - Export cell annotations as GeoJSON for visualization

## 1. Setup and Imports

In [1]:
import sys
from pathlib import Path

# Add src to path if running from notebooks directory
src_path = Path("../src").resolve()
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Analysis imports
from analysis import (
    # Feature extraction
    extract_features,
    extract_features_batch,
    # Preprocessing
    identify_marker_columns,
    scale_features,
    filter_markers,
    filter_cells,
    remove_outliers,
    # Clustering
    ClusteringConfig,
    to_anndata,
    run_clustering,
    clustering_pipeline,
    add_clusters_to_dataframe,
    # Visualization
    plot_marker_heatmap,
    plot_cluster_composition,
    plot_marker_distributions,
    plot_umap,
    plot_pca_variance,
    # Export
    export_to_geojson,
    export_mcmicro_batch,
)

# Pseudochannel imports (for finding experiments)
from pseudochannel import find_mcmicro_experiments

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Configure paths
# NOTE: Update these paths to match your data location

ROOT_PATH = Path("../data")  # Root directory containing MCMICRO experiments

# Or for a single experiment:
# MASK_PATH = Path("../data/experiment/segmentation/seg_mask.tif")
# IMAGE_PATH = Path("../data/experiment/background/image.ome.tiff")
# MARKER_PATH = Path("../data/experiment/markers.csv")

## 2. Feature Extraction

Extract morphology and marker intensity features from segmented cells.

### 2.1 Single Experiment

In [ ]:
# # Extract features from a single experiment
# # Uncomment and update paths to use:

# features_df = extract_features(
#     mask_path=MASK_PATH,
#     image_path=IMAGE_PATH,
#     marker_file=MARKER_PATH,
#     mcmicro_markers=True,
#     exclude_markers=["DAPI", "Background"],  # Markers to exclude
#     roi_name="experiment_1",  # Optional ROI identifier
# )
# features_df.head()

### 2.2 Batch Processing

In [ ]:
# # Batch extract features from all experiments
# # This finds experiments with segmentation masks and extracts features from each

# features_df = extract_features_batch(
#     root_path=ROOT_PATH,
#     segmentation_folder="segmentation",
#     mask_filename="seg_mask.tif",
#     output_folder="analysis",
#     output_filename="features.csv",
#     mcmicro_markers=True,
#     exclude_markers=["DAPI", "Background"],
#     save_individual=True,  # Save per-experiment CSVs
#     progress=True,
# )
# print(f"Extracted features for {len(features_df)} cells")

In [ ]:
# Or load pre-extracted features:
# features_df = pd.read_csv(ROOT_PATH / "combined_features.csv")

# For demonstration, create synthetic data
np.random.seed(42)
n_cells = 5000
n_markers = 20

features_df = pd.DataFrame({
    "ROI": np.random.choice(["sample_1", "sample_2", "sample_3"], n_cells),
    "label": np.arange(1, n_cells + 1),
    "centroid_x": np.random.uniform(0, 2000, n_cells),
    "centroid_y": np.random.uniform(0, 2000, n_cells),
    "area": np.random.lognormal(6, 0.5, n_cells),
})

# Add synthetic marker intensities
marker_names = [f"Marker_{i}" for i in range(1, n_markers + 1)]
for marker in marker_names:
    features_df[marker] = np.random.lognormal(0, 1, n_cells)

print(f"Features shape: {features_df.shape}")
features_df.head()

## 3. Preprocessing

Clean, filter, and normalize the feature data before clustering.

In [ ]:
# Identify marker columns automatically
marker_cols = identify_marker_columns(features_df)
print(f"Detected {len(marker_cols)} marker columns:")
print(marker_cols)

In [ ]:
# Filter cells by morphology (e.g., remove debris and large aggregates)
print(f"Before filtering: {len(features_df)} cells")

filtered_df = filter_cells(
    features_df,
    min_area=100,    # Minimum cell area
    max_area=5000,   # Maximum cell area
)
print(f"After area filtering: {len(filtered_df)} cells")

In [ ]:
# Remove outlier cells (optional)
filtered_df = remove_outliers(
    filtered_df,
    columns=marker_cols,
    method="percentile",
    lower=0.01,
    upper=0.99,
)
print(f"After outlier removal: {len(filtered_df)} cells")

In [ ]:
# Scale marker intensities
# Options: 'standard' (z-score), 'minmax' (0-1), 'log' (log1p)

scaled_df = scale_features(
    filtered_df,
    method="standard",
    columns=marker_cols,
)

print("Scaled marker statistics:")
scaled_df[marker_cols].describe().loc[["mean", "std"]].round(3)

## 4. Clustering

Perform dimensionality reduction and clustering to identify cell populations.

In [ ]:
# Configure clustering parameters
config = ClusteringConfig(
    n_pcs=30,                      # Number of principal components
    harmony_vars=["ROI"],          # Batch correction variables
    n_neighbors=15,                # Neighbors for graph construction
    louvain_resolution=1.0,        # Cluster resolution (higher = more clusters)
    random_state=42,
)

print(f"Clustering config: {config}")

In [ ]:
# Run the full clustering pipeline
# This performs: PCA -> Harmony batch correction -> Louvain clustering -> UMAP

adata = clustering_pipeline(
    scaled_df,
    config=config,
    marker_columns=marker_cols,
)

print(f"AnnData shape: {adata.shape}")
print(f"Number of clusters: {adata.obs['louvain'].nunique()}")
print(f"Cluster sizes:\n{adata.obs['louvain'].value_counts().sort_index()}")

In [ ]:
# Add cluster assignments back to the DataFrame
result_df = add_clusters_to_dataframe(
    scaled_df,
    adata,
    cluster_key="louvain",
    column_name="cluster",
)
result_df.head()

## 5. Visualization

Visualize clustering results and marker expression patterns.

### 5.1 PCA Variance

In [ ]:
# Plot PCA variance explained
plot_pca_variance(adata, n_pcs=20)

### 5.2 UMAP Embedding

In [ ]:
# UMAP colored by cluster
plot_umap(adata, color="louvain", title="UMAP - Louvain Clusters")

In [ ]:
# UMAP colored by ROI (to check batch correction)
plot_umap(adata, color="ROI", title="UMAP - By ROI")

### 5.3 Marker Heatmap

In [ ]:
# Marker expression heatmap by cluster
plot_marker_heatmap(
    adata,
    groupby="louvain",
    standard_scale="var",  # Normalize per marker
    cmap="viridis",
)

### 5.4 Cluster Composition

In [ ]:
# Cluster composition by ROI
plot_cluster_composition(
    result_df,
    cluster_col="cluster",
    group_col="ROI",
    normalize=True,
    title="Cluster Composition by Sample",
)

### 5.5 Marker Distributions

In [ ]:
# Marker distributions by cluster (first 8 markers)
plot_marker_distributions(
    result_df,
    markers=marker_cols[:8],
    hue="cluster",
    kind="box",
    ncols=4,
)

## 6. QuPath Export

Export cell annotations to GeoJSON format for visualization in QuPath.

In [ ]:
# # Export single experiment to GeoJSON
# # Uncomment and update paths to use:

# # Build cluster assignments dict for a specific ROI
# roi_df = result_df[result_df["ROI"] == "sample_1"]
# cluster_assignments = dict(zip(roi_df["label"].astype(int), roi_df["cluster"]))

# export_to_geojson(
#     mask_path=MASK_PATH,
#     output_path=Path("../output/annotations.geojson"),
#     cluster_assignments=cluster_assignments,
#     simplify_tolerance=1.0,  # Polygon simplification
#     alpha=128,  # Fill transparency
# )

In [ ]:
# # Batch export all experiments
# # Uncomment and update to use:

# exported = export_mcmicro_batch(
#     root_path=ROOT_PATH,
#     features_df=result_df,
#     cluster_col="cluster",
#     label_col="label",
#     roi_col="ROI",
#     output_folder="qupath",
#     output_filename="annotations.geojson",
#     progress=True,
# )
# print(f"Exported {len(exported)} GeoJSON files")

## 7. Save Results

In [ ]:
# # Save clustered features to CSV
# result_df.to_csv(ROOT_PATH / "clustered_features.csv", index=False)
# print("Saved clustered features")

In [ ]:
# # Save AnnData object (includes all embeddings and metadata)
# adata.write(ROOT_PATH / "analysis.h5ad")
# print("Saved AnnData object")

In [ ]:
# Save clustering config for reproducibility
import yaml

analysis_config = {
    "preprocessing": {
        "min_area": 100,
        "max_area": 5000,
        "outlier_method": "percentile",
        "outlier_lower": 0.01,
        "outlier_upper": 0.99,
        "scaling_method": "standard",
    },
    "clustering": {
        "n_pcs": config.n_pcs,
        "harmony_vars": config.harmony_vars,
        "n_neighbors": config.n_neighbors,
        "louvain_resolution": config.louvain_resolution,
        "random_state": config.random_state,
    },
}

# # Save config
# with open(ROOT_PATH / "analysis_config.yaml", "w") as f:
#     yaml.dump(analysis_config, f, default_flow_style=False)
# print("Saved analysis config")

# Display config
print(yaml.dump(analysis_config, default_flow_style=False))

## Summary

This notebook demonstrated the complete downstream analysis pipeline:

1. **Feature Extraction** - Extracted morphology and marker intensities from segmented cells
2. **Preprocessing** - Filtered cells by size, removed outliers, and scaled features
3. **Clustering** - Performed PCA, Harmony batch correction, and Louvain clustering
4. **Visualization** - Created UMAP plots, heatmaps, and composition charts
5. **Export** - Prepared GeoJSON files for QuPath visualization

### Output Files

After running on real data, the following outputs are created:

```
root/
├── experiment/
│   ├── analysis/
│   │   └── features.csv        # Per-cell features
│   └── qupath/
│       └── annotations.geojson # QuPath annotations
├── clustered_features.csv      # Combined features with clusters
├── analysis.h5ad               # AnnData with embeddings
└── analysis_config.yaml        # Analysis parameters
```